In [14]:
import json
import time
import boto3
import numpy as np
import random
import soundfile 
import sagemaker
import sagemaker.huggingface
import soundfile
import json

BUCKET="wav2vec2-music-recognition" # please use your bucket name
PREFIX = "huggingface-blog" 
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=BUCKET)

print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::767397693289:role/service-role/AmazonSageMaker-ExecutionRole-20240206T132494
sagemaker bucket: wav2vec2-music-recognition
sagemaker session region: ap-southeast-1


In [22]:
id = int(time.time())

In [23]:
from sagemaker.huggingface import HuggingFaceModel

huggingface_model = HuggingFaceModel(
        entry_point = 'inference.py',
        source_dir='./scripts',
        name = f'huggingface-wav2vec2-model-{id}',
        transformers_version='4.28.1', 
        pytorch_version='2.0.0', 
        py_version='py310',
        model_data="s3://wav2vec2-music-recognition/model.tar.gz",
        role=ROLE,
    )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [24]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge", 
    endpoint_name = f'huggingface-wav2vec2-endpoint-{id}'
)

Repacking model artifact (s3://wav2vec2-music-recognition/model.tar.gz), script artifact (./scripts), and dependencies ([]) into single tar.gz file located at s3://sagemaker-ap-southeast-1-767397693289/huggingface-wav2vec2-model-1709692846/model.tar.gz. This may take some time depending on model size...
Creating model with name: huggingface-wav2vec2-model-1709692846
Creating endpoint-config with name huggingface-wav2vec2-endpoint-1709692846
Creating endpoint with name huggingface-wav2vec2-endpoint-1709692846


-----------!

In [5]:
# inference audio file that download from S3 bucket or inference local audio file 

# s3.download_file(BUCKET, 'huggingface-blog/sample_audio/xxxxxx.wav', 'downloaded.wav')
# file_name ='downloaded.wav'

# download a sample audio file by using below link
!wget https://datashare.ed.ac.uk/bitstream/handle/10283/343/MKH800_19_0001.wav --no-check-certificate
    
file_name ='MKH800_19_0001.wav'

speech_array, sampling_rate = soundfile.read(file_name)

# ipd.Audio(data=np.asarray(speech_array), autoplay=True, rate=16000)

In [ ]:
%%time
json_request_data = {"speech_array": speech_array.tolist(),
                     "sampling_rate": sampling_rate}

from sagemaker.predictor import Predictor
endpoint_name = "huggingface-wav2vec2-endpoint-1709692846"
predictor = Predictor(endpoint_name=endpoint_name)

prediction = predictor.predict(json.dumps(json_request_data).encode('utf-8'))
print(prediction)

In [10]:
import json
x = json.loads(prediction.decode('utf-8'))
x = json.loads(x[0])
len(x)

768

In [22]:
np.array([x])

array([[-3.96828949e-02,  1.39496684e-01,  1.75963342e-01,
         1.04616731e-01,  3.71245921e-01, -2.85870016e-01,
         5.16653731e-02, -1.91958278e-01, -1.62618190e-01,
        -1.43105581e-01, -9.51759666e-02,  5.68000495e-01,
         1.56977713e-01,  2.86898464e-01,  7.05955252e-02,
        -6.03754781e-02,  3.77826303e-01,  1.15860447e-01,
        -1.04807869e-01, -6.06536448e-01, -8.36289451e-02,
        -7.98336938e-02,  1.33546859e-01,  4.96974707e-01,
         4.98156786e-01, -2.35502183e-01,  2.04559669e-01,
        -8.06412399e-02,  4.24592232e-04,  1.67548865e-01,
        -3.87298644e-01, -1.17105126e-01, -4.97059017e-01,
        -1.85206179e-02, -1.37613922e-01,  4.02108468e-02,
        -1.13016091e-01,  6.09809645e-02,  1.33838311e-01,
        -1.26640558e-01, -2.82535136e-01,  1.13820590e-01,
        -1.74130887e-01, -1.50130183e-01, -7.95352638e-01,
        -1.61202192e-01, -2.97009140e-01,  2.00484380e-01,
         1.02753565e-01, -1.70819595e-01, -5.38096912e-0

In [35]:
sampling_rate

48000

In [ ]:
import librosa
runtime = boto3.client('sagemaker-runtime', 'ap-southeast-1')
audio_array, sampling_rate = soundfile.read("Recording.mp3")
audio_array, sampling_rate = librosa.load("Recording.mp3", sr=16000)
json_request_data = {
    "speech_array": audio_array.tolist(),
    "sampling_rate": sampling_rate
    }


# response = runtime.invoke_endpoint(
#     EndpointName="huggingface-wav2vec2-endpoint-1709692846",
#     ContentType='application/json',
#     Body=json.dumps(json_request_data).encode('utf-8')
# )

from sagemaker.predictor import Predictor
endpoint_name = "huggingface-wav2vec2-endpoint-1709692846"
predictor = Predictor(endpoint_name=endpoint_name)

response = predictor.predict(json.dumps(json_request_data).encode('utf-8'))

res = response['Body'].read().decode('utf-8')

In [ ]:
res